# Manipulation.py: Bridge

In [1]:
import numpy as np
import robotic as ry
import manipulation as manip

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

for i in range(3):
	# Frame name
	box_name = f"box{i+1}"

	# Box color
	color = [0, 0, 0]
	color[i] = 1

	# Position
	position_val1 = 0.2 * i + .3

	C.addFrame(box_name) \
		.setPosition([position_val1, 0.05, 0.72]) \
		.setShape(ry.ST.ssBox, size=[0.04, 0.04, 0.12, 0.001]) \
		.setColor(color) \
		.setContact(1) \
		.setMass(.1)

C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
gripper = "l_gripper"
palm = "l_palm"
boxes = [f"box{i}" for i in range(1, 4)]
table = "table"

C.view()

0

In [4]:
from time import sleep

def build_bridge(target_position: list[float], vis: bool=False) -> bool:
	boxes = ["box1", "box2", "box3"]
	
	# Pick place waypoints
	M = manip.ManipulationModelling(C, helpers=[gripper])
	M.setup_pick_and_place_sequence(gripper, table, boxes)

	# Box 1
	grasp_direction = np.random.choice(["x", "y"])
	M.grasp_box(1., gripper, "box1", palm, grasp_direction)
	M.place_box(2., "box1", table, palm, "z")
	M.target_relative_xy_position(2., "box1", table, target_position)

	# Box 2
	grasp_direction = np.random.choice(["x", "y"])
	M.grasp_box(3., gripper, "box2", palm, grasp_direction)
	M.place_box(4., "box2", table, palm, "z")
	M.set_relative_distance(4., "box1", "box2", .05)

	# Box 3
	grasp_direction = np.random.choice(["x", "y"])
	M.grasp_box(5., gripper, "box3", palm, grasp_direction)
	place_direction = ry.FS.vectorX if grasp_direction == "y" else ry.FS.vectorY
	# M.komo.addObjective([6.], ry.FS.vectorZ, ["box3"], ry.OT.eq, [1e1], box3_dir)
	M.komo.addObjective([6.], place_direction, ["box3"], ry.OT.eq, [1e1], [0, 0, -1])
	M.komo.addObjective([6.], ry.FS.positionRel, ["box3", "box1"], ry.OT.sos, 1e1*np.array([[1, 0, 0],[0, 1, 0]]), 0)
	M.komo.addObjective([6.], ry.FS.positionRel, ["box3", "box2"], ry.OT.sos, 1e1*np.array([[1, 0, 0],[0, 1, 0]]), 0)
	M.set_relative_distance(6., "box3", "box1", .0)

	
	M.solve()

	keypoints = M.solve()
	if not M.feasible:
		# M.komo.report(plotOverTime=True)
		# C.view(True, f"{M.feasible}")
		return False, []
	
	# for i, k in enumerate(keypoints):
	# 	C.setJointState(k)
	# 	if i%2:
	# 		C.attach(table, boxes[i//2])
	# 	else:
	# 		C.attach(gripper, boxes[i//2])
	# 	C.view(False, f'step {i}')
	# 	sleep(1.)

	# return True, keypoints
	
	paths = []
	manip_models = []
	
	# Sub motions
	for i, box in enumerate(boxes):
		idx = i*2
		M1 = M.sub_motion(idx, accumulated_collisions=True)
		M1.retract([.0, .2], gripper)
		M1.approach([.8, 1.], gripper)
		path = M1.solve()
		if not M1.feasible:
			return False, []
		paths.append(path)
		manip_models.append(M1)

		M2 = M.sub_motion(idx+1, accumulated_collisions=True)
		path = M2.solve()
		if not M2.feasible:
			return False, []
		paths.append(path)
		manip_models.append(M2)

	# Visualization
	if vis:
		for i, m in enumerate(manip_models):
			m.play(C, 1.)
			if i%2:
				C.attach(table, boxes[i//2])
			else:
				C.attach(gripper, boxes[i//2])

	return True, paths

In [5]:
attempt_count = 30
success_count = 0
midpoint = [-0.105, 0.4, 0.745]

for l in range(attempt_count):
	housePosition = [midpoint[0] + np.random.random()*.2 -.1, midpoint[1] + np.random.random()*.2]
	success, _ = build_bridge(housePosition, vis=True)
	success_count += 1 if success else 0
	
print(f"Successful motions: {success_count}/{attempt_count}")
    

  -- infeasible:
     { time: 0.878523, evals: 186, done: 1, feasible: 0, sos: 0.301441, f: 0, ineq: 0.0667105, eq: 2.52535 }
-- pairCollision.cpp:libccd:354(0) WARNING: called MPR penetration for non intersecting meshes...
  -- infeasible:
     { time: 0.508312, evals: 163, done: 1, feasible: 0, sos: 0.292886, f: 0, ineq: 0.0983977, eq: 3.0065 }
-- pairCollision.cpp:libccd:359(0) WARNING: but GJK says intersection
-- pairCollision.cpp:libccd:354(0) WARNING: called MPR penetration for non intersecting meshes...
-- pairCollision.cpp:libccd:359(0) WARNING: but GJK says intersection
-- pairCollision.cpp:libccd:354(0) WARNING: called MPR penetration for non intersecting meshes...
-- pairCollision.cpp:libccd:359(0) WARNING: but GJK says intersection
-- pairCollision.cpp:libccd:354(0) WARNING: called MPR penetration for non intersecting meshes...
-- pairCollision.cpp:libccd:359(0) WARNING: but GJK says intersection
-- pairCollision.cpp:libccd:354(0) WARNING: called MPR penetration for non in